In [5]:
import os
import pandas as pd

script_dir = os.getcwd() 

print(f"Current working directory: {script_dir}")

Current working directory: c:\Users\willi\Documents\GitHub\Movie-Recommendation-System


Originally, the title.basics.tsv.gz file contained the following fields: tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, and genres.

Since the original title, start year, end year, and runtime are not relevant features, I have removed them. Additionally, I have filtered out any content that is not classified as a movie (excluding "short" films).

In [6]:
# Load data
basics_input_file = os.path.join(script_dir, "IMDB Datasets", "title.basics.tsv")
df = pd.read_csv(basics_input_file, sep="\t", dtype=str, na_values="\\N")

# Filter irrelevant columns
title_basics_df_filtered = df.loc[df['titleType'] == 'movie', ["tconst",  "primaryTitle", "isAdult", "startYear", "runtimeMinutes", "genres"]]

display(title_basics_df_filtered.head(10))

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,Miss Jerry,0,1894,45,Romance
144,tt0000147,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
498,tt0000502,Bohemios,0,1905,100,NaN
570,tt0000574,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,0,1907,90,Drama
610,tt0000615,Robbery Under Arms,0,1907,NaN,Drama
625,tt0000630,Hamlet,0,1908,NaN,Drama
668,tt0000675,Don Quijote,0,1908,NaN,Drama
672,tt0000679,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
828,tt0000838,A Cultura do Cacau,0,1909,NaN,NaN


To ensure data quality, I have also removed movies with fewer than 1,000 ratings, eliminating lesser-known films that could introduce noise into the recommendations.

In [7]:
# Load data
ratings_input_file = os.path.join(script_dir, "IMDB Datasets", "title.ratings.tsv") 
df = pd.read_csv(ratings_input_file, sep="\t", dtype=str, na_values="\\N")

df["numVotes"] = df["numVotes"].astype(int)

# Calculate Mean, Median, Mode
mean_votes = df["numVotes"].mean()
median_votes = df["numVotes"].median()
mode_votes = df["numVotes"].mode()[0]

# Print results
print(f"Mean: {mean_votes}")
print(f"Median: {median_votes}")
print(f"Mode: {mode_votes}")

ratings_df_filtered = df.loc[df["numVotes"] >= 1000]

display(ratings_df_filtered.head(10))

Mean: 1039.281932741051
Median: 27.0
Mode: 8


,tconst,averageRating,numVotes
0,tt0000001,5.7,2192
2,tt0000003,6.5,2297
4,tt0000005,6.2,3030
7,tt0000008,5.4,2374
9,tt0000010,6.8,8239
11,tt0000012,7.4,13908
12,tt0000013,5.7,2120
13,tt0000014,7.1,6365
14,tt0000015,6.2,1315
15,tt0000016,5.9,1707


To create a comprehensive dataset, I merged the filtered movie data with ratings and crew information using the common identifier tconst. Additionally, I removed the titleType and numVotes columns, as they are no longer needed.

In [8]:
# Load data
crew_input_file = os.path.join(script_dir, "IMDB Datasets", "title.crew.tsv")
df_crew = pd.read_csv(crew_input_file, sep="\t", dtype=str, na_values="\\N")

# Merge dataframes
df_merged = pd.merge(title_basics_df_filtered, ratings_df_filtered, on="tconst", how="inner")
df_merged = pd.merge(df_merged, df_crew, on="tconst", how="left")

# Remove numVotes
df_merged = df_merged.drop(columns=["numVotes"])

# Save data
output_file = os.path.join(script_dir, "Cleaned Datasets", "merged_movie_data.tsv")
df_merged.to_csv(output_file, sep="\t", index=False)

# Display
print(f"Merged dataset saved to: {output_file}")
display(df_merged.head(10))

Merged dataset saved to: c:\Users\willi\Documents\GitHub\Movie-Recommendation-System\Cleaned Datasets\merged_movie_data.tsv


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers
0,tt0000574,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,nm0846879,nm0846879
1,tt0002130,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy",7.1,"nm0078205,nm0655824,nm0209738",nm0019604
2,tt0002423,Passion,0,1919,113,"Biography,Drama,Romance",6.7,nm0523932,"nm0266183,nm0473134"
3,tt0002844,Fantômas: In the Shadow of the Guillotine,0,1913,54,"Crime,Drama",6.9,nm0275421,"nm0019855,nm0275421,nm0816232"
4,tt0003014,Ingeborg Holm,0,1913,96,Drama,7.0,nm0803705,"nm0472236,nm0803705"
5,tt0003037,Fantomas: The Man in Black,0,1913,61,"Crime,Drama",6.9,nm0275421,"nm0019855,nm0275421,nm0816232"
6,tt0003165,Fantômas: The Dead Man Who Killed,0,1913,90,"Crime,Drama,Mystery",6.9,nm0275421,"nm0019855,nm0275421,nm0816232"
7,tt0003419,The Student of Prague,0,1913,83,"Drama,Fantasy,Horror",6.4,"nm0263912,nm0753233","nm0263912,nm0210503"
8,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',0,1914,78,"Crime,Drama,Horror",6.4,nm0000428,"nm0000590,nm0000428"
9,tt0003740,Cabiria,0,1914,148,"Adventure,Drama,History",7.1,nm0665163,"nm0195339,nm0515385,nm0665163,nm0758215"
